### What is the derivative of a function?

The **derivative** tells us how much a function's output is changing for a given input change. Think back to using $\frac{rise}{run}$ in algebra class to calculate the slope of a line. Well, what if we had a function that did **not** trace a straight line? Imagine we had such a "curvy" function, and wanted to calculate its slope at a specific input point. Well, we would simply use the $\frac{rise}{run}$ formula and make _run_ infinitesimally small!

Here is the **limit definition of the derivative** from calculus, where $f(x)$ is our function of interest and $f'(x)$ its derivative. $\Delta$ indicates a change in $x$:

$$f'(x) = \lim_{\Delta x\to 0}\frac{f(x+\Delta x)-f(x)}{\Delta x}$$

**Note:** $y$ is used interchangeably with $f(x)$ later in the discussion.

Let's apply this to a real-world example. Say a person sets off walking from an origin point and picks up their pace with each passing second. Let $x$ represent time in seconds and the person's distance from the origin in meters be given by $f(x) = \frac{1}{3} x^2$

In [ ]:
# Import useful libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='whitegrid')
import pandas as pd

In [ ]:
# Plot the distance function
x = np.linspace(0,5,num=100)
f = lambda x: x**2/3
p = plt.plot(x, f(x))
xlab = plt.xlabel('Time (x)', fontsize=14)
ylab = plt.ylabel('Distance from Origin (m)', fontsize=14)
#plt.savefig('2_distance.png')

How might we find the **instantaneous rate of change** of our walker's position at 3 seconds? Using the power rule from calculus we can calculate the derivative of our function $f(x)$: $$f(x) = \frac{1}{3} x^2$$  
$$f'(x) = \frac{2}{3} x$$

Plugging in 3 for $x$ we get $f'(3) = 2$ _meters/second_. We can visualize the walker's unique pace at 3 seconds by adding a **tangent line** to our graph. The slope of the tangent line is equal to the derivative we just calculated at $x = 3$ and it intercepts our original function $f(x)$ **only** at $x = 3$.

In [ ]:
# Plot a line tangent to the distance function at x = 3
x = np.linspace(0,5,num=100)
f = lambda x: x**2/3
p = plt.plot(x, f(x))
s = plt.scatter(3, f(3))
t = lambda x: 2*x - 3
tang = plt.plot(x,t(x),ls='--')
plt.ylim(0,8)
xlab = plt.xlabel('Time (x)', fontsize=14)
ylab = plt.ylabel('Distance from Origin (m)', fontsize=14)
#plt.savefig('3_tangent.png')

### Why does it matter for machine learning?

To put derivatives in a machine learning context, let's imagine we are modeling the relationship between **x**: bank account balance, and **y**: account holder happiness. The relationship roughly follows a straight line with zero bias. Our machine learning model will have the form $y = w\cdot x$ and we'll use linear regression to train it to find the best weight $w$ for accurately predicting happiness units. 

**Aside**: Happiness units are measured from electrodes stuck to the heads of human volunteers. 

So our starting point is that we have a bunch of real-world examples of account balance ($x$) and corresponding happiness units ($y$) to help us. We'll call this the **training data**.

In [ ]:
# Plot the training data
x = np.linspace(-500,500, num=100)
eps = np.random.randn(100)*100
f = lambda x: 2*x + eps
y = f(x)
p = plt.scatter(x,y,edgecolor='k',s=20)
titl = plt.title('Training Data', fontsize=14)
xlab = plt.xlabel('Account Balance ($)', fontsize=14)
ylab = plt.ylabel('Happiness units (hu)', fontsize=14)

When we train our machine learning model, the goal will be to minimize a **cost function** that measures how far off the model's predictions are from reality. For this example we'll use the **quadtratic cost**, also called  **mean squared error (MSE)**: $$C(w) = \frac{1}{n}\Sigma_i(y_i - \hat{y}_i)^2$$

Where $y_i$ is the true happiness level, $\hat{y}_i$ the predicted happiness level given by $w\cdot x_i$ and $n$ the number of training examples. The cost function can thus be interpreted as the average squared error of the model, taken across the training data set. The derivative of the above cost function will be the key to optimizing _w_. First though, let's see how the quadratic cost behaves for a few example values of _w_.

In [ ]:
# Write a function for computing quadratic cost
def quad_cost(w, x, y):
    '''
    Inputs
        w: weight (scalar or vector)
        x: inputs (vector)
        y: outputs (vector)
    '''
    y_hat = np.dot(w,x.T)
    cost = np.mean(np.sum((y-y_hat)**2))
    return cost

In [ ]:
# Plot two example models.
# Visualize the residuals for each training data point.
# Display quadratic costs.

# Change w1 and w2 below to experiment with different values
w1 = 5
w2 = 1

fig, ax = plt.subplots(ncols=2, figsize=(12,6), sharex=True, sharey=True)

for k, w in zip([0,1],[w1,w2]):
    p = ax[k].scatter(x,y,edgecolor='k',s=20, label='True')
    xlab = ax[k].set_xlabel('Account Balance ($)', fontsize=14)
    ylab = ax[k].set_ylabel('Happiness units (hu)', fontsize=14)
    g1 = lambda x: w*x
    ax[k].plot(x,g1(x), label='Predicted')
    ax[k].set_title('w = %i' % w, fontsize=14)
    ax[k].set_xlim(-550,550)
    ax[k].set_ylim(-3000,3000)
    ax[k].legend()
    for i in range(len(x)):
        y2 = g1(x[i])
        ax[k].plot((x[i],x[i]),(y[i],y2), linewidth=1, color=sns.color_palette()[1])
    c = quad_cost(w,x,y)
    t = ax[k].text(-50,-2000,'C(%i) = %.1e $hu^2$' % (w,c), fontsize=16)

The lengths of the green lines connecting true values to predicted values represent the inputs $(y_i -\hat{y}_i)$ to the cost function. Looking at the above figures, you might have guessed that setting $w = 5$ resulted in much greater cost than $w=1$. Note that you can change the values for these weights in the code.

### Minimizing cost by hill descent (the 1-D precursor to gradient descent)

For the happiness problem we have only one parameter (_w_) to optimize with respect to the cost function. This makes it straightforward to calculate the derivative of the cost function and simply solve for its minimum analytically.  We could even plot quadratic cost as a function of _w_ and pretty much eyeball the solution...

In [ ]:
# Plot quadratic cost as a function of w
weights = np.linspace(1,3,num=1000)
errors = [quad_cost(w,x,y) for w in weights]
p = plt.plot(weights,errors)
xl = plt.xlabel('w', fontsize=14)
yl = plt.ylabel('Quadratic Cost', fontsize=14)

... but typical machine learning problems have many, many parameters which _all_ affect the cost and _all_ need optimizing. In such cases it is not reasonable to solve for the cost minimum analytically. Nor can you plot the relationship of parameters to cost for a high-dimensional problem.

So, for purpose of illustration we will optimize our single-parameter function by a process called **hill descent** which takes advantage of the cost derivative. This concept will serve as the basis of **gradient descent** for solving higher dimensional problems.

Say we randomly initialize _w_ to a value of 1.1. If we didn't know the explicit mathematical relationship between _w_ and _C(w)_, the best we could do is approximate the relationship by calculating the cost derivative at $w = 1.1$. Using the power rule and chain rule from calculus, the cost derivative with respect to _w_ is:$$\frac{dC}{dw} = \frac{-2}{n}\Sigma_i (y_i - \hat{y}_i)\cdot x_i$$

Plugging $w = 1.1$ into this equation (remember, $\hat{y}_i = w\cdot x_i$) gives us the **instantaneous rate of change** of $C(w)$ at that point. Since here it is a negative value, we know that _C(w)_ is decreasing as _w_ increases from its starting point of 1.1!

In [ ]:
# Write a function to calculate the cost derivative
def cost_deriv(w,x,y):
    '''
    Returns the gradient of the quadratic cost function with respect to w
    '''
    y_hat = np.dot(w,x)
    cost_deriv = np.mean(np.sum(2 * -x * (y-y_hat)))
    return cost_deriv

In [ ]:
# Plot C(w) and its tanget line at w = 1.1
p = plt.plot(weights,errors)
xl = plt.xlabel('w', fontsize=14)
yl = plt.ylabel('Quadratic Cost', fontsize=14)
p = plt.scatter(1.1,quad_cost(1.1,x,y))
slope = cost_deriv(1.1,x,y)
t = lambda w: slope*(w - 1.1) + quad_cost(1.1,x,y)
p = plt.plot(weights,t(weights), ls='--')
p = plt.ylim(0)

Armed with the cost derivative at the current value of _w_, we can iteratively nudge _w_ in the opposite direction until we reach an optimum. Here is some pseudo-code to describe the **update rule for hill descent**:

**while** not converged:
<p style="margin-left: 40px">$w^{(t+1)}=w^t-\eta \frac{dC}{dw}$</p>
<p style="margin-left: 40px">$t=t+1$</p>

...where $t$ is an iteration counter and $\eta$ the user-defined **learning rate**. The learning rate will be a small, positive number that ensures we move _w_ in small increments and don't overshoot the optimum. Convergence criteria can be defined by a max iteration count, for example, or by a minimum improvement in cost for a given update to _w_.

In [ ]:
# Iteratively nudge w in the opposite direction of the cost derivative

# Change the value of current_w to experiment with different initial values of w
current_w = 1.1

fig, ax = plt.subplots(nrows=2, ncols=3, figsize=(12,7))
for j in range(ax.shape[0]):
    for k in range(ax.shape[1]):
        # Plot C(w) function
        p = ax[j,k].plot(weights,errors)
        ax[j,k].scatter(current_w,quad_cost(current_w,x,y))
        # Calculate and plot tangent line
        slope = cost_deriv(current_w,x,y)
        t = lambda w: slope*(w - current_w) + quad_cost(current_w,x,y)
        p = ax[j,k].plot(weights,t(weights), ls='--')
        # Annotate subplot
        ax[j,k].text(1.7,.6e7,'$w = %.3f$' % current_w)
        xl = ax[j,k].set_xlabel('w', fontsize=14)
        yl = ax[j,k].set_ylabel('Quadratic Cost')
        ax[j,k].set_xlim(.9,3.1)
        ax[j,k].set_ylim(0,1e7)
        # Apply update rule for w
        current_w -= (1./23e6)*cost_deriv(current_w,x,y)

### Gradient Descent

How can we apply our hill descent technique to a higher-dimensional problem? Let's imagine we need to model happiness on three inputs instead of one: 
1. Account balance
2. Cups of coffee consumed
3. Local temperature (F)

In [ ]:
# Generate new dataset and display a few data points
bal = np.linspace(-500,500, num=100)
cof = np.random.randint(0,6,size=100)
temp = np.random.normal(loc=80, scale=15, size=100)
eps = np.random.randn(100)*100
w0, w1, w2, w3 = np.random.randn()*20, np.random.randint(1,4), \
                 np.random.randint(275,301), np.random.randint(-13,-7)
hu = w0 + w1*bal + w2*cof + w3*temp + eps
df = pd.DataFrame({'balance':bal,'cups_coffee':cof,'temp_F':temp})
df['hu'] = hu 
df = df.sample(frac=1)
df.reset_index(inplace=True,drop=True)
df.head().round(2)

In this problem we'll need to optimize a model of the form $$y=w_0+(w_1\cdot x_0)+(w_2\cdot x_1)+(w_3\cdot x_2)$$ 

where $w_0$ represents the bias, $w_1$ through $w_3$ the variable weights and $x_0$ through $x_2$ the input features. Our cost function is still the same as for 1-D hill descent:$$C(w) = \frac{1}{n}\Sigma_i(y_i - \hat{y}_i)^2$$

Let's also recall the cost derivative formula, since it will inform our calculation of the gradient: $$\frac{dC}{dw} = \frac{-2}{n}\Sigma_i (y_i - \hat{y}_i)\cdot x_i$$

The difference between hill descent and gradient descent is that now for each round of updates to the weights we'll need to compute the **partial derivative** of each weight with respect to $C(w)$. This means that as we cycle over the model weights to determine their updates, we treat all as constants _except_ for the one we are computing the partial derivative for right now. We store the partial derivatives in a vector called the **gradient** (notation: $\nabla$), which in this example will be a vector of length 4.


$\nabla C(w) = \frac{-2}{n} * \begin{align*}
\Big[
&\Sigma_i(y_i-\hat{y_i}), \\
&\Sigma_i(y_i-\hat{y_i})\cdot x_0, \\
&\Sigma_i(y_i-\hat{y_i})\cdot x_1, \\
&\Sigma_i(y_i-\hat{y_i})\cdot x_2
\Big]
\end{align*}
$

This means that in the pseudo-code below describing the weight updates, the subtraction term $w^t-\eta \nabla C(w)$ in fact implies the element-wise subraction of two vectors, each having length 4:

**while** not converged:
<p style="margin-left: 40px">$w^{(t+1)}=w^t-\eta \nabla C(w)$</p>
<p style="margin-left: 40px">$t=t+1$</p>

Let's use this knowledge to train a multi-dimensional model! First though we need to make a slight modification to the training data. Notice in the equation for $\nabla C(w)$ that the partial derivative for each weight is calculated by multiplying the error $(y_i-\hat{y_i})$ by the input feature $x_i$ that the weight interacts with. The bias term $w_0$ doesn't interact with any input feature, but we can say its input feature is **1** so that its partial derivative takes the same form as the others in the model.

In [ ]:
# Add a constant term to the training data
df['constant'] = 1
df = df[['constant','balance','cups_coffee','temp_F','hu']]
df.head().round(2)

Now we can write functions to calculate partial derivatives and perform gradient descent.

In [ ]:
# Write a function to compute feature partial derivative
def partial_deriv(errors, feature):
    derivative = -2 * np.mean(np.sum(errors*feature))
    return derivative

In [ ]:
def gradient_descent(features, target, init_weights, step_size, max_iter):
    iter_count = 0        # Initialize iteration counter at 0
    training_costs = []   # Initialize empty list to store costs during training
    converged = False
    weights = init_weights
    while not converged:
        # Compute predictions and errors
        predictions = np.dot(features, weights)
        errors = target - predictions
        # Update weights
        for i in range(len(weights)):
            deriv = partial_deriv(errors, features[:,i])
            weights[i] -= step_size * deriv
        # Compute cost and store
        current_cost = quad_cost(weights, features, target)
        training_costs.append(current_cost)
        # Increment the iteration count
        iter_count += 1
        if iter_count == max_iter:
            converged = True
    return(weights, training_costs)

In [ ]:
# Subset input features and target 'hu' values as numpy arrays
features = np.array(df[['constant','balance','cups_coffee','temp_F']]) 
target = np.array(df['hu'])
# Set remaining arguments for gradient_descent() function
init_weights = np.zeros(4)
step_size = 8e-8
max_iter = 10000
# Run gradient descent
weights, costs = gradient_descent(features, target, init_weights, step_size, max_iter)

Let's see how quadratic cost changed over training iterations:

In [ ]:
p = plt.plot(range(max_iter),costs)
xlab = plt.xlabel('Iteration', fontsize=14)
ylab = plt.ylabel('C(w)', fontsize=14)

When we generated the multi-dimensional dataset, we set the weights to be picked randomly from within specified ranges. Let's see how our learned weights stack up against the real weights.

In [ ]:
# Real weights
w1, w2, w3

In [ ]:
# Learned weights
weights[1:]

Let's also check how some model predictions stack up against the target values

In [ ]:
# Real hu values
target[:3]

In [ ]:
# Model predictions
np.dot(weights, features[:3,:].T)

### Summary

Having completed this tutorial, you now have a basic grasp on
    1. Derivatives and why they matter to machine learning
    2. Implementing hill descent to optimize a single parameter model
    3. Implementing gradient descent to optimize higher dimensional models
These skills and concepts can serve as your basis both for tackling interesting real-world problems and for learning and understanding more sophisticated machine learning techniques. Best of luck on your journey!